# TI3130: Data Wrangling Lab &mdash; Solutions (variant B)
**Julián Urbano &mdash; November 2021**

In [1]:
import sys
import numpy as np
import pandas as pd
print("python ", sys.version,
      "\nnumpy", np.__version__,
      "\npandas", pd.__version__)

python  3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)] 
numpy 1.21.4 
pandas 1.3.4


In [2]:
listings = pd.read_csv("airbnb_listings.csv")
reviews = pd.read_csv("airbnb_reviews.csv")

a) What hosts have listings in the Alexanderplatz `neighborhood` that do not require more than 2 nights?

In [47]:
listings \
    .query('neighborhood == "Alexanderplatz" & minimum_nights <= 2') \
    .loc[: , 'host_id'] \
    .drop_duplicates()

48        261953
90        632174
94        681637
98        722460
120       833209
          ...   
4590    36961901
4686    37597410
4701    29513589
4776    17698129
4987    39213221
Name: host_id, Length: 76, dtype: int64

b) How many users have submitted at least two reviews?

In [48]:
reviews \
    .groupby(['reviewer_id']) \
    .aggregate(reviews_per_user = ('reviewer_id', 'count')) \
    .query('reviews_per_user >= 2') \
    .count()

reviews_per_user    1031
dtype: int64

c) What reviewers have reviewed the same listing more than three times?

In [54]:
reviews \
    .groupby(['listing_id', 'reviewer_id'], as_index=False) \
    .aggregate(combination_count = ('reviewer_id', 'count')) \
    .query('combination_count > 3')

listing_id  reviewer_id  combination_count
6560       107968      2648035                  4
9544       171746     95627040                  4
11362      187206      5023231                  4
11459      187206     38158204                  4
15123      220808     30777232                  4
16293      230824     23863556                  4
16902      237038      1442940                  4
17172      238396     11379932                  4
29954      385059     38101555                  9
39369      613219    115045439                  4
43911      701909     13127535                  7
44974      719195     11354354                  4
46078      742211     23180684                  4

d) Select the top 10 hosts with most reviews. Show their `host_id`, `host_name` and number of `reviews`.

In [55]:
pd.merge(listings, reviews, how = 'inner', left_on = 'id', right_on = 'listing_id') \
    .groupby(['host_id', 'host_name'], as_index=False) \
    .aggregate(reviews = ('id_y', 'count')) \
    .sort_values('reviews', ascending=False) \
    .head(10)

host_id           host_name  reviews
63    286494           Project A      957
446  2547002            Studio A      900
33    163384        Alan & Kasia      848
238  1250154              Henrik      799
157   868282               Frank      783
21     98237              Damian      623
178   929943  Reineldis & Fabian      613
187   955509             Karsten      586
343  1654885            Claudius      573
112   632174           Stéphanie      443

e) Sort `neighborhood`s in descending order by the range of their prices (ie. maximum minus minimum). Consider only listings whose `price` is at least 20€/night, and show only neighborhoods with a range different from 0.

In [57]:
listings \
    .query('price >= 20') \
    .groupby('neighborhood', as_index=False) \
    .apply(lambda x: pd.Series({'range': x.price.max() - x.price.min()})) \
    .query('range != 0') \
    .sort_values('range', ascending=False)

neighborhood  range
44         Helmholtzplatz   4220
110  Tempelhofer Vorstadt   2480
83         Pankow Zentrum    776
113       Wedding Zentrum    630
71        Neue Kantstraße    580
..                    ...    ...
64            Marzahn-Süd      9
21            Buckow Nord      7
73      Niederschöneweide      6
76        Oberschöneweide      5
43        Hellersdorf-Süd      5

[114 rows x 2 columns]

f) Sort `neighborhoods` by their listing density, in descending order. Density may be calculated as the number of listings by surface. Surface may be calculated as the range of `latitude` multiplied by the range of `longitude`. Tip: rows with an `Inf` value may be removed by first [replacing](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) `np.inf` values with `np.nan` and then calling `dropna`.

In [58]:
listings \
    .groupby('neighborhood', as_index=False) \
    .apply(lambda x: pd.Series({
        'listings': x.id.count(),
        'range_lat': x.latitude.max() - x.latitude.min(), 
        'range_long': x.longitude.max() - x.longitude.min()
        })) \
    .assign(density = lambda x: x.listings / (x.range_lat * x.range_long)) \
    .replace(np.inf, np.nan) \
    .dropna() \
    .loc[:, ['neighborhood', 'density']] \
    .sort_values('density', ascending=False)

neighborhood       density
21                 Buckow Nord  2.447337e+07
117                     West 2  2.205573e+06
76             Oberschöneweide  1.931683e+06
38                  Hakenfelde  1.866623e+06
4    Alt-Hohenschönhausen Nord  1.852787e+06
..                         ...           ...
125        Zehlendorf  Südwest  4.587605e+03
60                   Mahlsdorf  3.595491e+03
51                   Kaulsdorf  2.729847e+03
120                     West 5  2.173805e+03
34              Gatow / Kladow  2.159298e+03

[116 rows x 2 columns]

g) Create a data frame where rows are `host_ids`, columns are `neighborhood_groups`, and values are the mean `price` of the listings of that host in that neighborhood group. Round the price to full euros.

In [59]:
nhood_per_host = listings \
    .groupby(['host_id', 'neighborhood_group'], as_index = False) \
    .aggregate(mean_price = ('price', 'mean')) \
    .round() \
    .pivot(index = 'host_id',
          columns = 'neighborhood_group', values = 'mean_price')

nhood_per_host

neighborhood_group  Charlottenburg-Wilm.  Friedrichshain-Kreuzberg  \
host_id                                                              
2217                                 NaN                       NaN   
2986                                 NaN                       NaN   
3718                                 NaN                       NaN   
4108                                 NaN                       NaN   
11015                                NaN                      49.0   
...                                  ...                       ...   
39219660                             NaN                      78.0   
39222511                             NaN                       NaN   
39230948                             NaN                       NaN   
39257044                             NaN                       NaN   
39259413                             NaN                      70.0   

neighborhood_group  Lichtenberg  Marzahn - Hellersdorf  Mitte  Neukölln  \
host_id                                                                   
2217                        NaN                    NaN   60.0       NaN   
2986                        NaN                    NaN    NaN       NaN   
3718                        NaN                    NaN    NaN       NaN   
4108                        NaN                    NaN    NaN       NaN   
11015                       NaN                    NaN    NaN       NaN   
...                         ...                    ...    ...       ...   
39219660                    NaN                    NaN    NaN       NaN   
39222511                    NaN                    NaN    NaN       NaN   
39230948                    NaN                    NaN   87.0       NaN   
39257044                    NaN                    NaN    NaN       NaN   
39259413                    NaN                    NaN    NaN       NaN   

neighborhood_group  Pankow  Reinickendorf  Spandau  Steglitz - Zehlendorf  \
host_id                                                                     
2217                   NaN            NaN      NaN                    NaN   
2986                  17.0            NaN      NaN                    NaN   
3718                  90.0            NaN      NaN                    NaN   
4108                   NaN            NaN      NaN                    NaN   
11015                  NaN            NaN      NaN                    NaN   
...                    ...            ...      ...                    ...   
39219660               NaN            NaN      NaN                    NaN   
39222511               NaN           74.0      NaN                    NaN   
39230948               NaN            NaN      NaN                    NaN   
39257044               NaN            NaN      NaN                    NaN   
39259413               NaN            NaN      NaN                    NaN   

neighborhood_group  Tempelhof - Schöneberg  Treptow - Köpenick  
host_id                                                         
2217                                   NaN                 NaN  
2986                                   NaN                 NaN  
3718                                   NaN                 NaN  
4108                                  26.0                 NaN  
11015                                  NaN                 NaN  
...                                    ...                 ...  
39219660                               NaN                 NaN  
39222511                               NaN                 NaN  
39230948                               NaN                 NaN  
39257044                             240.0                 NaN  
39259413                               NaN                 NaN  

[4507 rows x 12 columns]

h) Same as g), but show only cases where the average price is cheaper than 30€/night.

In [39]:
listings \
    .groupby(['host_id', 'neighborhood_group'], as_index = False) \
    .aggregate(mean_price = ('price', 'mean')) \
    .round() \
    .query('mean_price < 30') \
    .pivot(index = 'host_id', columns = 'neighborhood_group', values = 'mean_price')

neighborhood_group  Charlottenburg-Wilm.  Friedrichshain-Kreuzberg  \
host_id                                                              
2986                                 NaN                       NaN   
4108                                 NaN                       NaN   
12723                                NaN                       NaN   
130019                               NaN                       NaN   
159734                               NaN                       NaN   
...                                  ...                       ...   
39137222                             NaN                       NaN   
39152545                             NaN                       NaN   
39156665                             NaN                       NaN   
39199345                             NaN                       NaN   
39216851                             NaN                      25.0   

neighborhood_group  Lichtenberg  Marzahn - Hellersdorf  Mitte  Neukölln  \
host_id                                                                   
2986                        NaN                    NaN    NaN       NaN   
4108                        NaN                    NaN    NaN       NaN   
12723                       NaN                    NaN    NaN       NaN   
130019                      NaN                    NaN    NaN       NaN   
159734                      NaN                    NaN    NaN       NaN   
...                         ...                    ...    ...       ...   
39137222                    NaN                    NaN   25.0       NaN   
39152545                    NaN                    NaN    NaN       NaN   
39156665                    NaN                    NaN    NaN       NaN   
39199345                   17.0                    NaN    NaN       NaN   
39216851                    NaN                    NaN    NaN       NaN   

neighborhood_group  Pankow  Reinickendorf  Spandau  Steglitz - Zehlendorf  \
host_id                                                                     
2986                  17.0            NaN      NaN                    NaN   
4108                   NaN            NaN      NaN                    NaN   
12723                 16.0            NaN      NaN                    NaN   
130019                 NaN            NaN      NaN                    NaN   
159734                 NaN            NaN      NaN                    NaN   
...                    ...            ...      ...                    ...   
39137222               NaN            NaN      NaN                    NaN   
39152545               NaN            NaN      NaN                    NaN   
39156665              28.0            NaN      NaN                    NaN   
39199345               NaN            NaN      NaN                    NaN   
39216851               NaN            NaN      NaN                    NaN   

neighborhood_group  Tempelhof - Schöneberg  Treptow - Köpenick  
host_id                                                         
2986                                   NaN                 NaN  
4108                                  26.0                 NaN  
12723                                  NaN                 NaN  
130019                                25.0                 NaN  
159734                                25.0                 NaN  
...                                    ...                 ...  
39137222                               NaN                 NaN  
39152545                              22.0                 NaN  
39156665                               NaN                 NaN  
39199345                               NaN                 NaN  
39216851                               NaN                 NaN  

[609 rows x 12 columns]

i) Take the result of g) and turn it into long format. Drop any rows with missing values.

In [42]:
nhood_per_host \
    .reset_index() \
    .melt(id_vars = 'host_id', var_name = 'neighborhood_group', value_name = 'mean_price') \
    .dropna()

host_id    neighborhood_group  mean_price
47       156897  Charlottenburg-Wilm.        46.0
73       221653  Charlottenburg-Wilm.        61.0
81       261488  Charlottenburg-Wilm.        84.0
84       264072  Charlottenburg-Wilm.        55.0
90       276540  Charlottenburg-Wilm.        70.0
...         ...                   ...         ...
53877  36314853    Treptow - Köpenick        50.0
53902  36762019    Treptow - Köpenick        50.0
53933  37375563    Treptow - Köpenick        65.0
53948  37553259    Treptow - Köpenick        20.0
54059  39034992    Treptow - Köpenick        35.0

[4567 rows x 3 columns]